In [1]:
%load_ext autoreload
%autoreload 2


import torch

import torch.nn as nn
from torch.distributions import Categorical, Poisson, MixtureSameFamily
from matplotlib import pyplot as plt
import pandas as pd

# Cd to code
import os
import sys
os.chdir('/cluster/home/kheuto01/code/prob_diff_topk')
sys.path.append('/cluster/home/kheuto01/code/prob_diff_topk')

from datasets import example_datasets, to_numpy
import os
import torch
import numpy as np
import pandas as pd
import time
from functools import partial
from distributions import QuantizedNormal
from torch_models import  MixtureOfTruncNormModel, torch_bpr_uncurried, deterministic_bpr, SpatialWaves
#from torch_perturb.torch_pert_topk import PerturbedTopK
from metrics import top_k_onehot_indicator
from torch_training import train_epoch_largesynth
from torch_perturb.perturbations import perturbed
from torch_models import NegativeBinomialRegressionModel, MixtureOfTruncNormModel, torch_bpr_uncurried, deterministic_bpr

2024-10-16 14:47:58.064990: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-16 14:47:58.115792: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-16 14:47:58.115827: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-16 14:47:58.117164: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-16 14:47:58.125687: I tensorflow/core/platform/cpu_feature_guar

In [2]:

def generate_synthetic_data(num_locations=10, num_time_points=50, num_fixed_effects=3, seed=42):
    torch.manual_seed(seed)
    np.random.seed(seed)

    # Generate time points
    time = torch.linspace(0, 1, num_time_points).unsqueeze(1).expand(-1, num_locations)

    # Generate fixed effects
    X = torch.randn(num_time_points, num_locations, num_fixed_effects)

    # Set true parameter values
    true_beta_0 = 1.0
    true_beta = torch.tensor([0.5, -0.3, 0.7])
    true_sigma_0 = 0.2
    true_sigma_1 = 0.1
    true_rho = 0.3
    true_theta = 5.0

    # Generate random effects
    cov_matrix = torch.tensor([
        [true_sigma_0**2, true_rho * true_sigma_0 * true_sigma_1],
        [true_rho * true_sigma_0 * true_sigma_1, true_sigma_1**2]
    ])
    random_effects = torch.distributions.MultivariateNormal(
        loc=torch.zeros(2),
        covariance_matrix=cov_matrix
    ).sample((num_locations,))
    b_0 = random_effects[:, 0]
    b_1 = random_effects[:, 1]

    # Calculate log_mu
    fixed_effects = true_beta_0 + torch.einsum('tli,i->tl', X, true_beta)
    random_intercepts = b_0.expand(num_time_points, -1)
    random_slopes = b_1.expand(num_time_points, -1)
    log_mu = fixed_effects + random_intercepts + random_slopes * time

    # Generate y using the NegativeBinomial distribution
    mu = torch.exp(log_mu)
    y = torch.distributions.NegativeBinomial(
        total_count=true_theta,
        probs=true_theta / (true_theta + mu)
    ).sample()

    return X, y, time, {
        'beta_0': true_beta_0,
        'beta': true_beta,
        'sigma_0': true_sigma_0,
        'sigma_1': true_sigma_1,
        'rho': true_rho,
        'theta': true_theta,
        'b_0': b_0,
        'b_1': b_1
    }

In [3]:
X, y, time, true_params = generate_synthetic_data()

print("X shape:", X.shape)
print("y shape:", y.shape)
print("time shape:", time.shape)
print("\nTrue parameters:")
for key, value in true_params.items():
    print(f"{key}: {value}")

X shape: torch.Size([50, 10, 3])
y shape: torch.Size([50, 10])
time shape: torch.Size([50, 10])

True parameters:
beta_0: 1.0
beta: tensor([ 0.5000, -0.3000,  0.7000])
sigma_0: 0.2
sigma_1: 0.1
rho: 0.3
theta: 5.0
b_0: tensor([-0.2883, -0.3926, -0.1699, -0.0410,  0.1375, -0.2222, -0.1980, -0.1431,
         0.1103,  0.2116])
b_1: tensor([-0.1037, -0.1209, -0.0921, -0.0807,  0.0954, -0.3297, -0.0871, -0.0660,
         0.2698,  0.0177])


In [4]:
# Initialize the model
num_locations, num_fixed_effects = X.shape[1], X.shape[2]
model = NegativeBinomialRegressionModel(num_locations, num_fixed_effects)

In [5]:
model.rho.data = torch.tensor([0.02], dtype=torch.float32, requires_grad=True)

In [6]:
model.log_likelihood(y, X, time)

tensor(-15.0671, grad_fn=<AddBackward0>)

In [7]:
cov = model.get_covariance_matrix()

In [8]:
assert cov.requires_grad

In [9]:
def calculate_log_likelihood_gradients(model, X, y, time):
    # Ensure gradients are being tracked
    for param in model.parameters():
        param.requires_grad_(True)
    
    # Forward pass to compute log-likelihood
    log_likelihood = model.log_likelihood(y, X, time)
    
    # Compute gradients
    log_likelihood.backward()
    
    # Store gradients in a dictionary
    gradients = {}
    for name, param in model.named_parameters():
        if param.grad is not None:
            gradients[name] = param.grad.clone()
        else:
            gradients[name] = torch.zeros_like(param)
    
    return gradients

# Assuming you have already initialized the model and generated synthetic data
# model = NegativeBinomialRegressionModel(...)
# X, y, time, true_params = generate_synthetic_data(...)

# Calculate gradients
gradients = calculate_log_likelihood_gradients(model, X, y, time)

In [10]:
gradients

{'beta_0': tensor([5.3865]),
 'beta': tensor([-3.1868,  2.5952, -2.8444]),
 'b_0': tensor([1.1078, 0.4994, 0.4833, 0.6514, 0.2354, 0.3962, 0.3264, 0.7953, 0.3361,
         0.5203]),
 'b_1': tensor([ 0.3960,  0.3276, -0.0294,  0.3470,  0.4248,  0.6551, -0.3307,  0.2029,
          0.3313,  0.8706]),
 'log_sigma_0': tensor([0.1867]),
 'log_sigma_1': tensor([2.4988]),
 'rho': tensor([-0.4146]),
 'log_theta': tensor([-5.8018])}

In [8]:
# Create a single tensor with all the true parameters
true_params_tensor = torch.cat([
    torch.tensor([true_params['beta_0']]),
    true_params['beta'],
    true_params['b_0'],
    true_params['b_1'],
    torch.log(torch.tensor([true_params['sigma_0']])),
    torch.log(torch.tensor([true_params['sigma_1']])),
    torch.atanh(torch.tensor([true_params['rho']])),
    torch.log(torch.tensor([true_params['theta']]))
])

# Update the model parameters with the true values
model.update_params(true_params_tensor)

In [14]:
model.log_likelihood(y, X, time)

tensor(-7.0503, grad_fn=<AddBackward0>)